#Der Kohlendioxidgehalt in Hessen Teil 1

Die Rohdaten stellen uns vor folgenden Aufgaben:
1. Den Datentyp müssen wir von Objekt in Zahlen umwandeln.
2. Die Datumspalte und Zeitspalte in ein Datumsformat bringen ("%d-%m-%y %H:%M").

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/data'


messwerteb.txt	messwertec.txt	messwerted.txt


Pandas ist ein Framework.  Hauptbestandteil ist die Klasse DataFrame, mit der sich zweidimensionale Tabellen, die aus Zeilen und Spalten bestehen, aufbereiten und umarbeiten lassen.

Wir laden die pandas Bibliothek und kürzen sie mit pd ab.




In [ ]:
import pandas as pd

Die Datei *messwerte.txt*  liefert uns Hinweise, wie wir die Datei in Python laden.

**Dezimaltrennzeichen**

Deutschland verwendet ein Komma (,) als **Dezimaltrennzeichen**.

**Feldtrennzeichen** 

Die meisten europäischen Länder, trennen mit ';'' anstatt ',' und das Dezimaltrennzeichen ist ',' statt '.'
 
Die Messdaten einlesen (pd.read_csv) und die Zellen mit ';' trennen. Dezimaltrennzeichen ',' ( sep=';' , decimal=',').


Datum;Zeit;Kohlendioxid (CO2)[mg/m³];Temperatur[°C];Luftdruck[hPa]

01.12.2019;01:00;867;-3,5;1023

01.12.2019;02:00;885;-3,6;1023



In [ ]:
df= pd.read_csv('/content/drive/MyDrive/data/messwerteb.txt', sep=';' ,decimal=',')

Mit der head-Methode erhalten wir eine Übersicht über die ersten fünf Zeilen. 

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa]
0,01.12.2019,01:00,867,"-3,5",1023
1,01.12.2019,02:00,885,"-3,6",1023
2,01.12.2019,03:00,890,"-3,2",1022
3,01.12.2019,04:00,883,"-3,4",1022
4,01.12.2019,05:00,891,"-3,6",1022


Beim Einlesen von Daten versucht Pandas, die Daten automatisch in ein Zahlenformat (Integer oder Floats) zu konvertieren. Mit df.dtypes erhalten wir den entsprechenden Datentyp. Für die Messparameter und das Datum ist das nicht gelungen. Für die Verarbeitungsschritte, die uns noch bevorstehen, ist dieses Format ungeeignet. 

In [ ]:
df.dtypes

Datum                        object
Zeit                         object
Kohlendioxid (CO2)[mg/m³]    object
Temperatur[°C]               object
Luftdruck[hPa]               object
dtype: object

Wir tauschen die Dezimaltrennzeichen in Spalte Temperatur[°C]  ',' mit '.' und erstellen eine neue Spalte Temperatur (df['Temperatur']).

In [ ]:

df['Temperatur'] = [x.replace(',', '.') for x in df['Temperatur[°C]']]

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur
0,01.12.2019,01:00,867,"-3,5",1023,-3.5
1,01.12.2019,02:00,885,"-3,6",1023,-3.6
2,01.12.2019,03:00,890,"-3,2",1022,-3.2
3,01.12.2019,04:00,883,"-3,4",1022,-3.4
4,01.12.2019,05:00,891,"-3,6",1022,-3.6


Wir wandeln die Datenreihe Luftdruck[hPa], Temperatur und Kohlendioxid (CO2)[mg/m³] von object zu float um, und erstellen neue Spalten namens Luftdruck, Temperatur und Kohlendioxid.

WICHTIG: errors='coerce' wandelt '-' und alle weiteren Zeichen zu NaN um.

In [ ]:
df['Luftdruck'] = pd.to_numeric(df['Luftdruck[hPa]'], errors='coerce')

df['Temperatur'] = pd.to_numeric(df['Temperatur'], errors='coerce')

df['Kohlendioxid'] = pd.to_numeric(df['Kohlendioxid (CO2)[mg/m³]'], errors='coerce')

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0


In [ ]:
df.tail()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid
4147,21.05.2020,20:00,-,-,-,NaN,NaN,NaN
4148,21.05.2020,21:00,-,-,-,NaN,NaN,NaN
4149,21.05.2020,22:00,-,-,-,NaN,NaN,NaN
4150,21.05.2020,23:00,-,-,-,NaN,NaN,NaN
4151,21.05.2020,24:00,-,-,-,NaN,NaN,NaN


In [ ]:
df.dtypes

Datum                         object
Zeit                          object
Kohlendioxid (CO2)[mg/m³]     object
Temperatur[°C]                object
Luftdruck[hPa]                object
Temperatur                   float64
Luftdruck                    float64
Kohlendioxid                 float64
dtype: object

Für die weitere Verarbeitung der Daten brauchen wir ein bestimmtes **Datumsformat**. Zum Beispiel für FP Prophet.

Wir addieren zur Spalte 'Zeit' :00 und fügen eine neue Spalte 
'Sekundenformat' hinzu, ohne Sekundenformat gibt es eine Fehlermeldung, z.B. bei FP Prophet.


In [ ]:
df['Sekundenformat'] = df['Zeit'] + ':00'

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid,Sekundenformat
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0,01:00:00
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0,02:00:00
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0,03:00:00
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0,04:00:00
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0,05:00:00


Dieser Weg verursachte einen Fehler: 
Monat und Tag zu Beginn richtig aber zum Schluß falsch:
df['Datum2']=pd.to_datetime(df.Datum) + pd.to_timedelta(df.period)

df["Date"] = df["Datum2"].apply(lambda x: datetime.combine(x, datetime.min.time()))

df["Date"] = df["Datum"].dt.strftime("%d-%m-%y %H:%M")

Wir nutzen die Pandas-Funktion **to_datetime**. Um datumsspezifische Operationen durchführen zu können.

In [ ]:
df['DatumYDM']=pd.to_datetime(df.Datum)

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid,Sekundenformat,DatumYDM
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0,01:00:00,2019-01-12
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0,02:00:00,2019-01-12
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0,03:00:00,2019-01-12
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0,04:00:00,2019-01-12
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0,05:00:00,2019-01-12


In [ ]:
df.dtypes

Datum                                object
Zeit                                 object
Kohlendioxid (CO2)[mg/m³]            object
Temperatur[°C]                       object
Luftdruck[hPa]                       object
Temperatur                          float64
Luftdruck                           float64
Kohlendioxid                        float64
Sekundenformat                       object
DatumYDM                     datetime64[ns]
dtype: object


Wir informieren Pandas, dass die zu lesenden Werte nacheinander Angaben zum Tag, Monat, Jahr beinhalten. 

In [ ]:
df["DatumMDY"] = df["DatumYDM"].dt.strftime("%d-%m-%y")

Wir wandeln Spalte Sekundenformat in 0 days 01:00:00 um. (zählt nur Stunden bis 24 und beginnt bei 0 Tage)

In [ ]:
df['Stundenzaehler']=pd.to_timedelta(df.Sekundenformat)

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid,Sekundenformat,DatumYDM,DatumMDY,Stundenzaehler
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0,01:00:00,2019-01-12,12-01-19,0 days 01:00:00
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0,02:00:00,2019-01-12,12-01-19,0 days 02:00:00
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0,03:00:00,2019-01-12,12-01-19,0 days 03:00:00
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0,04:00:00,2019-01-12,12-01-19,0 days 04:00:00
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0,05:00:00,2019-01-12,12-01-19,0 days 05:00:00


Wir addieren zur Spalte DatumMDY die Spalte Stundenzaehler = 2019-12-01 01:00:00

In [ ]:
df['DatumStundenzaehler']=pd.to_datetime(df.DatumMDY) + pd.to_timedelta(df.Stundenzaehler)

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid,Sekundenformat,DatumYDM,DatumMDY,Stundenzaehler,DatumStundenzaehler
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0,01:00:00,2019-01-12,12-01-19,0 days 01:00:00,2019-12-01 01:00:00
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0,02:00:00,2019-01-12,12-01-19,0 days 02:00:00,2019-12-01 02:00:00
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0,03:00:00,2019-01-12,12-01-19,0 days 03:00:00,2019-12-01 03:00:00
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0,04:00:00,2019-01-12,12-01-19,0 days 04:00:00,2019-12-01 04:00:00
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0,05:00:00,2019-01-12,12-01-19,0 days 05:00:00,2019-12-01 05:00:00



Das Datumsformat wird in 01-12-2019 01:00 umgwandelt

In [ ]:
df["DatumFinal"] = df["DatumStundenzaehler"].dt.strftime("%d-%m-%y %H:%M")

In [ ]:
df.head()

,Datum,Zeit,Kohlendioxid (CO2)[mg/m³],Temperatur[°C],Luftdruck[hPa],Temperatur,Luftdruck,Kohlendioxid,Sekundenformat,DatumYDM,DatumMDY,Stundenzaehler,DatumStundenzaehler,DatumFinal
0,01.12.2019,01:00,867,"-3,5",1023,-3.5,1023.0,867.0,01:00:00,2019-01-12,12-01-19,0 days 01:00:00,2019-12-01 01:00:00,01-12-19 01:00
1,01.12.2019,02:00,885,"-3,6",1023,-3.6,1023.0,885.0,02:00:00,2019-01-12,12-01-19,0 days 02:00:00,2019-12-01 02:00:00,01-12-19 02:00
2,01.12.2019,03:00,890,"-3,2",1022,-3.2,1022.0,890.0,03:00:00,2019-01-12,12-01-19,0 days 03:00:00,2019-12-01 03:00:00,01-12-19 03:00
3,01.12.2019,04:00,883,"-3,4",1022,-3.4,1022.0,883.0,04:00:00,2019-01-12,12-01-19,0 days 04:00:00,2019-12-01 04:00:00,01-12-19 04:00
4,01.12.2019,05:00,891,"-3,6",1022,-3.6,1022.0,891.0,05:00:00,2019-01-12,12-01-19,0 days 05:00:00,2019-12-01 05:00:00,01-12-19 05:00


Wir löschen die folgenden Spalten:

In [ ]:
df=df.drop(columns=['Datum', 'Zeit','Sekundenformat','Kohlendioxid (CO2)[mg/m³]','Temperatur[°C]','Luftdruck[hPa]','DatumYDM','DatumMDY','Stundenzaehler','DatumStundenzaehler'])

In [ ]:
df.head()

,Temperatur,Luftdruck,Kohlendioxid,DatumFinal
0,-3.5,1023.0,867.0,01-12-19 01:00
1,-3.6,1023.0,885.0,01-12-19 02:00
2,-3.2,1022.0,890.0,01-12-19 03:00
3,-3.4,1022.0,883.0,01-12-19 04:00
4,-3.6,1022.0,891.0,01-12-19 05:00


Wir nennen Spalte DatumFinal in Datum um

In [ ]:
df=df.rename(columns={'DatumFinal': 'Datum'})

In [ ]:
df.head()

,Temperatur,Luftdruck,Kohlendioxid,Datum
0,-3.5,1023.0,867.0,01-12-19 01:00
1,-3.6,1023.0,885.0,01-12-19 02:00
2,-3.2,1022.0,890.0,01-12-19 03:00
3,-3.4,1022.0,883.0,01-12-19 04:00
4,-3.6,1022.0,891.0,01-12-19 05:00


Wir setzen Spalte Datum als Index, dadurch können wir die  Vorzüge der Datumsklasse nutzen.


In [ ]:
df=df.set_index('Datum')

In [ ]:
df.head()

,Temperatur,Luftdruck,Kohlendioxid
Datum,,,
01-12-19 01:00,-3.5,1023.0,867.0
01-12-19 02:00,-3.6,1023.0,885.0
01-12-19 03:00,-3.2,1022.0,890.0
01-12-19 04:00,-3.4,1022.0,883.0
01-12-19 05:00,-3.6,1022.0,891.0


Das Ergebnis: Wir haben das Datum als nutzbares  Datumsformat umgewandelt. 
Die Temperatur, Luftdruck und Kohlendioxid sind im Zahlenformat und NaN Werte für fehlende Daten.

In [ ]:
df

,Temperatur,Luftdruck,Kohlendioxid
Datum,,,
01-12-19 01:00,-3.5,1023.0,867.0
01-12-19 02:00,-3.6,1023.0,885.0
01-12-19 03:00,-3.2,1022.0,890.0
01-12-19 04:00,-3.4,1022.0,883.0
01-12-19 05:00,-3.6,1022.0,891.0
...,...,...,...
21-05-20 20:00,NaN,NaN,NaN
21-05-20 21:00,NaN,NaN,NaN
21-05-20 22:00,NaN,NaN,NaN


In [ ]:
 df.dtypes

Temperatur      float64
Luftdruck       float64
Kohlendioxid    float64
dtype: object